In [37]:
import os
from gpuutils import GpuUtils
GpuUtils.allocate(gpu_count=1, framework='keras')

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True) 

GPU 0  will be allocated


In [38]:
#import sys
#sys.path.insert(0,'/home/halin/.local/lib')
from matplotlib import pyplot as plt
import numpy as np
#import seaborn as sns
import pandas as pd
from gc import callbacks
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, UpSampling1D
from keras_flops import get_flops
import Help_functions.creating_models as cm
import Help_functions.plot_functions as pf
import Help_functions.data_manage as dm
from Model_classes.NewPhysicsAutoencoder import NewPhysicsAutoencoder
from Model_classes.SecondCNNModel import SecondCNNModel
from Model_classes.DenseModel import DenseModel
from Model_classes.ConvAutoencoder import ConvAutoencoder



In [39]:
data_url = '/home/halin/Autoencoder/Data/'
x_test, y_test, smask_test, signal, noise, std, mean = dm.load_data(all_signals=True,
																																		 data_path=data_url, 
																																		 small_test_set=1000,
																																		 number_of_files=10)


In [18]:
x = x_test[0]
print(x_test.shape)

(364782, 100, 1)


In [ ]:
save_path = '/home/halin/Autoencoder/Models/test_models'
model_path = '/home/halin/Autoencoder/Models/CNN_122/CNN_122_model_1.h5'

pf.plot_performance(path=model_path, x_test=x_test,smask_test=smask_test,save_path=save_path,std=std,mean=mean)


In [42]:
model_path = '/home/halin/Autoencoder/Models/CNN_122/CNN_122_model_1.h5'
model = load_model(model_path)
for i, layer in enumerate(model.layers):
  print(i, layer)
  try:
    print("     ", layer.activation)
  except AttributeError:
    print('     no activation attribute')  

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f4de21c2760>
     no activation attribute
1 <tensorflow.python.keras.layers.convolutional.Conv1D object at 0x7f4de21c2ee0>
      <function relu at 0x7f4ec2b1c3a0>
2 <tensorflow.python.keras.layers.pooling.MaxPooling1D object at 0x7f4de7185a00>
     no activation attribute
3 <tensorflow.python.keras.layers.convolutional.Conv1D object at 0x7f4de7185a60>
      <function relu at 0x7f4ec2b1c3a0>
4 <tensorflow.python.keras.layers.pooling.MaxPooling1D object at 0x7f4de7293e80>
     no activation attribute
5 <tensorflow.python.keras.layers.convolutional.Conv1D object at 0x7f4de7293340>
      <function relu at 0x7f4ec2b1c3a0>
6 <tensorflow.python.keras.layers.pooling.MaxPooling1D object at 0x7f4de722a9a0>
     no activation attribute
7 <tensorflow.python.keras.layers.core.Flatten object at 0x7f4de722a970>
     no activation attribute
8 <tensorflow.python.keras.layers.core.Dense object at 0x7f4de722a310>
      <function linear